### <a id='overview'>1. Overview</a>

#### Motivation
- Use advanced SQL technique to answer some questions regarding the NBA 
- Some answers match successfully with independent reports

#### Method
- SQL technique applied
    - common table expression (CTE)
    - row_number() over()
    - rank() over()
    - left join, inner join
    - union all
    - min(), max(), round(), sum(), avg(), abs(), count()
    - alias for columns and tables
    - case ... when ...
    - group_concat()
- Python libraries used
    - pandas
    - sqlite3
    - os


### <a id='sam'>2. About the author: Kam Leung Yeung (Sam)</a>

* PhD in Cognitive Psychology, Iowa State University in Ames, Iowa, USA

**Social media**:

* [LinkedIn](https://www.linkedin.com/in/kamleungyeung/)
* [Google Scholar](https://scholar.google.com/citations?user=OwUmaN8AAAAJ)
* [GitHub](https://github.com/k-l-yeung)
* [Tableau](https://public.tableau.com/app/profile/kam.leung.yeung#!/)


**Data source**
- [NBA game data from kaggle](https://www.kaggle.com/datasets/nathanlauga/nba-games?resource=download)

### <a id='toc'>3. Table of content</a>

1. <a href='#overview'>Overview</a>  
2. <a href='#sam'>About the author: Kam Leung Yeung</a> 
3. <a href='#toc'>Table of content</a> 
4. <a href='#eda'>Read in data</a>  
5. <a href='#df_to_sql'>Read pandas dataframe into sql database</a>  
6. <a href='#q'>Questions to be answered</a>  
 - <a href='#40'>Most **consecutive** 40+ pts game by a player</a>  
 - <a href='#top3'>The top 3 individual player score in every season between 2012 and 2022</a>  
 - <a href='#top5'>The top 5 score individual player score since 2004</a>  
 - <a href='#blown'>The biggest blown out in every season and the corresponding margin</a>  
 - <a href='#versatile'>Identify versatile players who started with most different positions</a>  



### <a id='eda'>Read in data</a>

In [1]:
import pandas as pd
import os
import sqlite3 as db

In [2]:
os.chdir('C:/Users/siu_s/Documents/0_portfolio/kaggle_nba/archive')

In [3]:
df_d = pd.read_csv('games_details.csv')
df_p = pd.read_csv('players.csv')
df_g = pd.read_csv('games.csv')
df_t = pd.read_csv('teams.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645953 entries, 0 to 645952
Data columns (total 29 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            645953 non-null  int64  
 1   TEAM_ID            645953 non-null  int64  
 2   TEAM_ABBREVIATION  645953 non-null  object 
 3   TEAM_CITY          645953 non-null  object 
 4   PLAYER_ID          645953 non-null  int64  
 5   PLAYER_NAME        645953 non-null  object 
 6   NICKNAME           30362 non-null   object 
 7   START_POSITION     247215 non-null  object 
 8   COMMENT            105602 non-null  object 
 9   MIN                540350 non-null  object 
 10  FGM                540350 non-null  float64
 11  FGA                540350 non-null  float64
 12  FG_PCT             540350 non-null  float64
 13  FG3M               540350 non-null  float64
 14  FG3A               540350 non-null  float64
 15  FG3_PCT            540350 non-null  float64
 16  FT

In [5]:
df_d.head(2)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22101005,1610612750,MIN,Minnesota,1630162,Anthony Edwards,Anthony,F,NaN,36:22,...,0.0,8.0,8.0,5.0,3.0,1.0,1.0,1.0,15.0,5.0
1,22101005,1610612750,MIN,Minnesota,1630183,Jaden McDaniels,Jaden,F,NaN,23:54,...,2.0,4.0,6.0,0.0,0.0,2.0,2.0,6.0,14.0,10.0


In [6]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7228 entries, 0 to 7227
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PLAYER_NAME  7228 non-null   object
 1   TEAM_ID      7228 non-null   int64 
 2   PLAYER_ID    7228 non-null   int64 
 3   SEASON       7228 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 226.0+ KB


In [7]:
df_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     25796 non-null  object 
 1   GAME_ID           25796 non-null  int64  
 2   GAME_STATUS_TEXT  25796 non-null  object 
 3   HOME_TEAM_ID      25796 non-null  int64  
 4   VISITOR_TEAM_ID   25796 non-null  int64  
 5   SEASON            25796 non-null  int64  
 6   TEAM_ID_home      25796 non-null  int64  
 7   PTS_home          25697 non-null  float64
 8   FG_PCT_home       25697 non-null  float64
 9   FT_PCT_home       25697 non-null  float64
 10  FG3_PCT_home      25697 non-null  float64
 11  AST_home          25697 non-null  float64
 12  REB_home          25697 non-null  float64
 13  TEAM_ID_away      25796 non-null  int64  
 14  PTS_away          25697 non-null  float64
 15  FG_PCT_away       25697 non-null  float64
 16  FT_PCT_away       25697 non-null  float6

In [8]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LEAGUE_ID           30 non-null     int64  
 1   TEAM_ID             30 non-null     int64  
 2   MIN_YEAR            30 non-null     int64  
 3   MAX_YEAR            30 non-null     int64  
 4   ABBREVIATION        30 non-null     object 
 5   NICKNAME            30 non-null     object 
 6   YEARFOUNDED         30 non-null     int64  
 7   CITY                30 non-null     object 
 8   ARENA               30 non-null     object 
 9   ARENACAPACITY       26 non-null     float64
 10  OWNER               30 non-null     object 
 11  GENERALMANAGER      30 non-null     object 
 12  HEADCOACH           30 non-null     object 
 13  DLEAGUEAFFILIATION  30 non-null     object 
dtypes: float64(1), int64(5), object(8)
memory usage: 3.4+ KB


<a href='#toc'>Back to table of content</a>

### <a id='df_to_sql'>Read pandas dataframe into sql database</a>

[Getting an Excel CSV into a SQLITE database using Python](https://youtu.be/6ArtqEmw49g)

In [9]:
# will create the example.db file

# alternative: sqlite3.connect(':memory:')
# what it does is that everything is run on RAM
conn = db.connect('example.db')

In [10]:
# allow us to execute sql commands
c = conn.cursor()

In [11]:
# write records in a df to a sql db
df_t.to_sql(
    name = 'team',
    con = conn,
    if_exists = 'replace',
    index = False
)

In [12]:
# write records in a df to a sql db
df_g.to_sql(
    name = 'game',
    con = conn,
    if_exists = 'replace',
    index = False
)

In [13]:
# write records in a df to a sql db
df_d.to_sql(
    name = 'detail',
    con = conn,
    if_exists = 'replace',
    index = False
)

In [14]:
df_p.to_sql(
    name = 'player',
    con = conn,
    if_exists = 'replace',
    index = False,
    dtype = {
        'PLAYER_NAME': 'text',
        'TEAM_ID': 'real',
        'PLAYER_ID': 'real',
        'SEASON': 'real'
    }
)

In [15]:
# commit to the changes made
conn.commit()

# when the db is no longer used
# conn.close()

<a href='#toc'>Back to table of content</a>

### <a id='q'>Questions answered with SQL</a>

#### <a id='40'>Most consecutive 40+ pts game by a player</a>

In [16]:
query = '''

/* create a table with */
with a as (
select 
    t1.nickname as team, 
    g.game_date_est as game_date,
    t1.team_id
from
    team t1 inner join game g on t1.team_id = g.home_team_id
    
union all

select 
    t1.nickname as team, 
    g.game_date_est as game_date,
    t1.team_id
from
    team t1 inner join game g on t1.team_id = g.VISITOR_TEAM_ID
),

/* add row number */
b as (
select *, row_number() over(partition by team order by game_date) as game_no 
from a
),

/* set the target streak record*/
c as (
select 
    team, 
    game_no,
    player_name, 
    game_no - row_number() over(partition by player_name order by game_no) as sub,
    game_date, 
    fg3m, 
    pts
from 
    b inner join detail d on d.TEAM_ID = b.team_id
    inner join game g on d.game_id = g.game_id
where 
    g.GAME_DATE_EST = b.game_date and
    pts > 40
order by 
    player_name, game_no 
),

/* details about the streak */
dd as (
select 
    team, 
    player_name, 
    count(*) as times, 
    min(game_date) as strek_start,
    max(game_date) as streak_end,
    round(avg(pts),1) as streak_points_avg
from c
group by 
    player_name, sub
order by 
    times desc
)

select * from dd
limit 5
'''

In [17]:
pd.read_sql(query, conn)

,team,player_name,times,strek_start,streak_end,streak_points_avg
0,Rockets,James Harden,5,2018-12-25,2019-01-03,42.8
1,Lakers,Kobe Bryant,5,2007-03-16,2007-03-25,53.6
2,Lakers,Kobe Bryant,4,2006-01-06,2006-01-11,46.0
3,Lakers,Kobe Bryant,4,2006-03-31,2006-04-07,44.8
4,Thunder,Russell Westbrook,4,2017-02-26,2017-03-03,44.3


The list of answer matches reports in [Business Insider](https://www.businessinsider.com/james-harden-40-point-streak-huge-price-rockets-2019-1#:~:text=James%20Harden%20is%20on%20one%20of%20the%20hottest%20runs%20in,NBA%20history%20to%20do%20so.) (Harden's record) and [ESPN](https://www.espn.com/blog/statsinfo/post/_/id/130075/russell-westbrook-hits-historic-marks-with-another-40-point-game-in-loss-to-suns) (Westbrook's record)

<a href='#toc'>Back to table of content</a>

#### <a id='top3'>The top 3 individual player score in every season between 2012 and 2022</a>

In [18]:
query = '''
with a as 
(select 
    season, 
    rank() over(partition by season order by pts desc) as rk, 
    player_name, 
    pts, 
    t1.nickname as home, t2.nickname as away, game_date_est as game_date
from 
    detail d inner join game g on d.game_id = g.game_id
    inner join team t1 on g.home_team_id = t1.team_id
    inner join team t2 on g.VISITOR_TEAM_ID = t2.team_id

order by 
    season, rk, player_name
)

select * 
from a 
where 
    rk <=3 and season between 2012 and 2022
'''

In [19]:
pd.read_sql(query, conn)

,season,rk,player_name,pts,home,away,game_date
0,2012,1,Stephen Curry,54.0,Knicks,Warriors,2013-02-27
1,2012,2,Kevin Durant,52.0,Mavericks,Thunder,2013-01-18
2,2012,3,Carmelo Anthony,50.0,Heat,Knicks,2013-04-02
3,2013,1,Carmelo Anthony,62.0,Knicks,Hornets,2014-01-24
4,2013,2,LeBron James,61.0,Heat,Hornets,2014-03-03
5,2013,3,Kevin Durant,54.0,Thunder,Warriors,2014-01-17
6,2014,1,Kyrie Irving,57.0,Spurs,Cavaliers,2015-03-12
7,2014,2,Kyrie Irving,55.0,Cavaliers,Trail Blazers,2015-01-28
8,2014,3,Russell Westbrook,54.0,Pacers,Thunder,2015-04-12
9,2015,1,Kobe Bryant,60.0,Lakers,Jazz,2016-04-13


<a href='#toc'>Back to table of content</a>

#### <a id='top5'>The top 5 score individual player score since 2004</a>

In [20]:
query = '''
select 
    season, player_name, pts, t1.nickname as home, t2.nickname as away, game_date_est as game_date
from 
    detail d inner join game g on d.game_id = g.game_id
    inner join team t1 on g.home_team_id = t1.team_id
    inner join team t2 on g.VISITOR_TEAM_ID = t2.team_id
order by 
    pts desc
limit 5
'''

In [21]:
pd.read_sql(query, conn)

,SEASON,PLAYER_NAME,PTS,home,away,game_date
0,2005,Kobe Bryant,81.0,Lakers,Raptors,2006-01-22
1,2016,Devin Booker,70.0,Celtics,Suns,2017-03-24
2,2006,Kobe Bryant,65.0,Lakers,Trail Blazers,2007-03-16
3,2020,Stephen Curry,62.0,Warriors,Trail Blazers,2021-01-03
4,2005,Kobe Bryant,62.0,Lakers,Mavericks,2005-12-20


<a href='#toc'>Back to table of content</a>

#### <a id='blown'>The biggest blown out in every season and the corresponding margin</a>

In [22]:
query = '''
select 
    season,  
    t1.nickname as home, t2.nickname as away,
    case
        when pts_home > pts_away then 'home team won'
        else 'road team won'
    end as winner,
    max(abs(pts_home-pts_away)) as pts_diff, 
    game_date_est
from 
    detail d inner join game g on d.game_id = g.game_id
    inner join team t1 on g.home_team_id = t1.team_id
    inner join team t2 on g.VISITOR_TEAM_ID = t2.team_id
group by 
    season
order by 
    season

'''

In [23]:
pd.read_sql(query, conn)

,SEASON,home,away,winner,pts_diff,GAME_DATE_EST
0,2003,Grizzlies,Nets,home team won,47.0,2003-12-13
1,2004,Mavericks,Rockets,home team won,40.0,2005-05-07
2,2005,Suns,Trail Blazers,home team won,45.0,2005-12-06
3,2006,76ers,Rockets,road team won,50.0,2007-03-18
4,2007,Nuggets,Thunder,home team won,52.0,2008-03-16
5,2008,Pelicans,Nuggets,road team won,58.0,2009-04-27
6,2009,Knicks,Mavericks,road team won,50.0,2010-01-24
7,2010,Lakers,Cavaliers,home team won,55.0,2011-01-11
8,2011,Trail Blazers,Hornets,home team won,44.0,2012-02-01
9,2012,Thunder,Pelicans,home team won,45.0,2013-02-27


"(2021) Memphis Grizzlies beat OKC Thunder by 73 points" reported in [USA Today](https://www.usatoday.com/story/sports/2021/12/03/grizzlies-thunder-blowout-most-lopsided-wins-sports/8853268002/)

<a href='#toc'>Back to table of content</a>

#### <a id='versatile'>Identify versatile players who started with most different positions</a>

In [24]:
query = '''
select 
    t.nickname as team,
    player_name as player,
    count(distinct start_position) as total,
    group_concat(distinct start_position) as positions
from 
    detail d inner join team t on d.TEAM_ID = t.team_id
group by 
    player_name
having 
    count(distinct start_position) >=3
order by 
    team, player
'''

In [25]:
pd.read_sql(query, conn)

,team,player,total,positions
0,76ers,Ben Simmons,3,"G,C,F"
1,Bucks,Giannis Antetokounmpo,3,"F,G,C"
2,Bucks,Pat Connaughton,3,"G,F,C"
3,Bulls,Linton Johnson,3,"F,G,C"
4,Bulls,Tim Thomas,3,"F,C,G"
5,Cavaliers,Jarell Martin,3,"C,F,G"
6,Clippers,Marcus Morris Sr.,3,"F,C,G"
7,Clippers,Nicolas Batum,3,"F,C,G"
8,Clippers,Robert Covington,3,"F,C,G"
9,Clippers,Terance Mann,3,"G,F,C"


<a href='#toc'>Back to table of content</a>